# Aerosol calibration

Use the AR6 per-species ERFari calibrations, from Chapter 6 Fig. 6.12.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import json
import scipy.stats

from tqdm import tqdm

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')

In [ ]:
emitted_species = [
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
]

In [ ]:
concentration_species = [
    'CH4', 'N2O',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee'
]

In [ ]:
species_out = {}
for ispec, species in enumerate(emitted_species):
    species_rcmip_name = species.replace("-", "")
    emis_in = df_emis.loc[
        (df_emis['Scenario']=='ssp245') & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_emis['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    species_out[species] = emis_in
    
for ispec, species in enumerate(concentration_species):
    species_rcmip_name = species.replace("-", "")
    conc_in = df_conc.loc[
        (df_conc['Scenario']=='ssp245') & (df_conc['Variable'].str.endswith("|"+species_rcmip_name)) & 
        (df_conc['Region']=='World'), '1750':'2100'
    ].interpolate(axis=1).values.squeeze()
    species_out[species] = conc_in

In [ ]:
species_df = pd.DataFrame(species_out, index=range(1750,2101))
species_df

In [ ]:
species_df.loc[2019,'HFC-125':]

In [ ]:
radiative_efficiency = pd.Series({
    'HFC-125': 0.23378,
    'HFC-134a': 0.16714,
    'HFC-143a': 0.168,
    'HFC-152a': 0.10174,
    'HFC-227ea': 0.27325,
    'HFC-23': 0.19111,
    'HFC-236fa': 0.25069,
    'HFC-245fa': 0.24498,
    'HFC-32': 0.11144,
    'HFC-365mfc': 0.22813,
    'HFC-4310mee': 0.35731
})

In [ ]:
hfc_erf = (species_df.loc[2019,'HFC-125':] * radiative_efficiency/1000)
hfc_sum = hfc_erf.sum()

In [ ]:
erfari_emitted = pd.Series({
    'Sulfur': -0.234228,
    'BC': 0.144702,
    'OC': -0.072143,
    'NH3': -0.033769,
    'NOx': -0.009166,
    'VOC': -0.002573,
    'CO': 0,
    'CH4': -0.002653,
    'N2O': -0.00209,
    'HFC-125': -0.00808 * hfc_erf['HFC-125']/hfc_sum,
    'HFC-134a':-0.00808 * hfc_erf['HFC-134a']/hfc_sum,
    'HFC-143a':-0.00808 * hfc_erf['HFC-143a']/hfc_sum,
    'HFC-152a':-0.00808 * hfc_erf['HFC-152a']/hfc_sum,
    'HFC-227ea':-0.00808 * hfc_erf['HFC-227ea']/hfc_sum,
    'HFC-23':-0.00808 * hfc_erf['HFC-23']/hfc_sum,
    'HFC-236fa':-0.00808 * hfc_erf['HFC-236fa']/hfc_sum,
    'HFC-245fa':-0.00808 * hfc_erf['HFC-245fa']/hfc_sum,
    'HFC-32':-0.00808 * hfc_erf['HFC-32']/hfc_sum,
    'HFC-365mfc':-0.00808 * hfc_erf['HFC-365mfc']/hfc_sum,
    'HFC-4310mee':-0.00808 * hfc_erf['HFC-4310mee']/hfc_sum
})
erfari_emitted

In [ ]:
# erfari radiative efficiency per Mt or ppb or ppt
re = erfari_emitted / (species_df.loc[2019, :] - species_df.loc[1750, :])

In [ ]:
re

In [ ]:
scalings = scipy.stats.uniform.rvs(np.minimum(re*2, 0), np.maximum(re*2, 0)-np.minimum(re*2, 0), size=(50000, 20), random_state=3729329)

In [ ]:
samples = 50000

In [ ]:
erfari = np.zeros((351, samples))

for i in tqdm(range(samples)):
    erfari[:, i] = ((species_df - species_df.loc[1750])*scalings[i,:]).sum(axis=1)

In [ ]:
pl.plot(erfari);
#pl.ylim(-4, 0.5);

In [ ]:
np.sum(np.isnan(erfari))

In [ ]:
pl.hist(erfari[269, :])
np.percentile(erfari[269, :].mean(axis=0), (5, 16, 50, 84, 95))

In [ ]:
df = pd.DataFrame(scalings, columns=[
        'Sulfur',
        'BC',
        'OC',
        'NH3',
        'NOx',
        'VOC',
        'CO',
        'CH4',
        'N2O',
        'HFC-125',
        'HFC-134a',
        'HFC-143a',
        'HFC-152a',
        'HFC-227ea',
        'HFC-23',
        'HFC-236fa',
        'HFC-245fa',
        'HFC-32',
        'HFC-365mfc',
        'HFC-4310mee'
    ]
)

In [ ]:
df

In [ ]:
df.to_csv('../data/parameter_sets/erfari.csv', index=False)